Imports

In [15]:
import pipefy
import pandas as pd
import numpy as np
from datetime import datetime
import json

Set pipe ID to get data from Pipefy

In [16]:
pipe = pipefy.Pipe("302551116")

Get cards data from Pipefy

In [17]:
data = pipe.get_data()

Turn the data returned from GraphQL into various dataframes

In [18]:
#['df_cards', 'df_current_phase', 'df_assignees', 'df_comments', 'df_labels', 'df_phases_history', 'df_fields']
dfs = pipe.set_dataframes(data)

Turn the column value in df_fields from string into a list type and then explode the list.

In [19]:
dfs["df_fields"] = pd.eval(f"value2 = dfs['df_fields']['value'].str.strip('[\"\"]').str.split('\", \"')", target=dfs["df_fields"])
dfs["df_fields"] = dfs["df_fields"].explode("value2", ignore_index=True)
dfs["df_fields"].drop(columns=["value"], inplace=True)

Set the column phase in df_phases_history from dict into various columns

In [20]:
dfs["df_phases_history"] = pd.concat([dfs["df_phases_history"], dfs["df_phases_history"]["phase"].apply(pd.Series)], axis=1)
dfs["df_phases_history"].drop(columns=["phase"], inplace=True)

Turn all the dataframes into CSV files.

In [21]:
for df in dfs:
    dfs[df].to_csv(f"../datasets/data_{df[3:len(df)]}.csv", index = False)
